# JN pipeline

# Introduction

Jupyter Notebook to prepare flow cytofluometry data before input. The conversion will take the images extracted from the .cif file using the IFC package, convert to a multichannel image and add metadata to the .tif header.
Furthermore, it is possible to import images into OMERO using ezomero and add the masks

####  Import python libraries

In [ ]:
import os
import uuid
import json
import getpass

import numpy as np
import tifffile
import ezomero

from omero.gateway import BlitzGateway
from tifffile import TiffFile
from skimage import filters
from skimage.io import imread
from ome_types import to_xml, OME
from ome_types.model import Image, Pixels, Channel, TiffData

####  Functions to create the multichannel tiff and associate mask into OMERO


In [ ]:
def multi_channel_conversion(files_path, metadata_path, files_output):
    with open(metadata_path, 'r') as file:
        metadata = json.load(file)
    channels = metadata['wavelenght']
    channels = [item for sublist in channels for item in sublist]
    print(channels)
    os.chdir(files_path)
    files = sorted(os.listdir(files_path), key=str.lower)
    try:
        len(files) % len(channels) != 0
    except NameError:
        print(f"Something wrong with the file number, not multiples of {len(channels)}")
    else:
        os.mkdir(files_output)
        block_size = len(channels)
        array_list = []
        round = 0
        for i in range(0, len(files), block_size):
            block_files = files[i:i + block_size]
            array_list.clear()
            for im in block_files:
                img = tifffile.imread(im)
                array = np.rint((np.asarray(img) / 256) * 65535).astype(np.uint16)
                array_list.append(array)
                with TiffFile(f'{im}') as tif:
                    sample = tif.pages[0]
                    dtype = sample.dtype
                    pixel = tif.pages[0].tags["SamplesPerPixel"].value
                    width = tif.pages[0].tags["ImageWidth"].value
                    lenght = tif.pages[0].tags["ImageLength"].value
                    bits = tif.pages[0].tags["BitsPerSample"].value
            xml_channel_list = []
            stacked_array = np.stack(array_list, axis=2)
            transposed_array = np.transpose(stacked_array, (2, 0, 1))
            file_name = block_files[0]
            file_name = file_name[:-10]
            for count, value in enumerate(channels):
                if value == 0:
                    chn = Channel(
                        id=f"Channel:{round}:{count}",
                        acquisition_mode="BrightField",
                        samples_per_pixel = f"{pixel}"
                    )
                    xml_channel_list.append(chn)
                else:
                    chn = Channel(
                        id=f"Channel:{round}:{count}",
                        acquisition_mode="FluorescenceCorrelationSpectroscopy",
                        fluor="Natural",
                        excitation_wavelength= int(value),
                        excitation_wavelength_unit="nm"
                    )
                    xml_channel_list.append(chn)
            ome = OME(
                uuid="urn:uuid:" + str(uuid.uuid4()),
                creator="tifffile --v tifffile 2023.7.10")
            tfd = TiffData(
                first_c=0,
                first_t=0,
                first_z=0,
                ifd=0,
                plane_count=1,
                uuid=TiffData.UUID(
                    file_name=f"{file_name}.tiff",
                    value="urn:uuid:" + str(uuid.uuid4())
                ))
            img = Image(
                id=f"Image:{round}",
                name=f"{file_name}.tiff",
                pixels=Pixels(
                    id=f"Pixels:{round}",
                    channels=xml_channel_list,
                    tiff_data_blocks=[tfd],
                    type=f"{dtype}",
                    dimension_order="XYCTZ",
                    significant_bits = f"{bits}",
                    size_x=f"{width}",
                    size_y=f"{lenght}",
                    size_z=1,
                    size_c= f"{len(channels)}",
                    size_t=1)
            )
            ome.images.append(img)
            ome = to_xml(ome, validate=True)
            tifffile.imwrite(f"{files_output}/{file_name}.ome.tiff",
                             transposed_array,
                             dtype=transposed_array.dtype,
                             shape=transposed_array.shape,
                             metadata={'axes': 'CYX'},
                             description= ome)
            round += 1


def associate_mask(conn, dts, path, channels):
    dataset = conn.getObject("Dataset", dts)
    ID = []
    for image in dataset.listChildren():
        ID.append(image.getId())
    mask_folder_path = path
    os.chdir(mask_folder_path)
    files = os.listdir(mask_folder_path)
    for im in ID:
        print(f"Processing Image with ID:{im}")
        image = conn.getObject("Image", im)
        channel = 0
        for i in files:
            if channel <= channels:
                img = imread(path)
                # Apply thresholding to convert the grayscale img to binary
                threshold = filters.threshold_otsu(img)
                binary_image = img >= threshold
                # Convert binary img to binary array
                binary_array = (binary_image / 255).astype(np.int64)
                mask = omero_rois.mask_from_binary_image(binary_array, rgba=(255, 0, 0, 150), z=0, c=channel, t=0,
                                                         text=f"Channel_{channel}", raise_on_no_mask=False)
                create_roi(image, [mask])
                channel += 1
                os.remove(i)
            else:
                files = files[channels:]
                print(f"Finish mask upload with ID:{im}")
                break

# Step 1) Extract images and metadata

The R script can be run in the Jupyter Notebook thanks to the package rpy2 
(https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba)

In [ ]:
path_to_R = input("Specify the path to R:")
os.environ['R_HOME'] = f"{path_to_R}"
from rpy2.robjects.packages import IFC
from rpy2.robjects.packages import jsonlite

setwd("/working_directory_where_cif_files_are")

## get exemplary data from IFCdata - Change accordingly
test_data = system.file(package = "IFCdata", "extdata", "example.cif")
## test_data = "path_to_your_data"

#EXTRACT OFFSETS
## extracts offsets of the IFDs (Image Field Directories) within a XIF file. 
cif_offs <- getOffsets(fileName = test_data, fast = FALSE)

#EXTRACT METADATA
## get metadata: Retrieves rich information from RIF, CIF and DAF files.
metadata <- getInfo(fileName = test_data, from = "analysis")
metadata$illumination$Channel <- 1:nrow(metadata$illumination)
## extract metadata and save it as json file
wavelenght <- as.list(metadata$illumination$wavelength)
metadata_flat <- list(number_objects = metadata$objcount,
                      date = metadata$date,
                      instrument = metadata$instrument,
                      brightfield = metadata$brightfield,
                      illumination = metadata$illumination,
                      images = metadata$Images,
                      mask = metadata$masks,
                      wavelenght = wavelenght)
json_data <- toJSON(metadata_flat, pretty = TRUE)
write(json_data, file = "metadata.json")

#Extract images
## 1) count the number of objects
nobj <- as.integer(metadata$objcount)

## 2) OPTIONAL: subset objects from the .cif file
sel <- sample(0:(nobj-1), min(5, nobj))
sub_offs <- subsetOffsets(cif_offs, objects = sel, image_type = "img")

# 3) extracts IFDs (Image File Directory) in RIF or CIF files.
## IFDs contain information about images or masks of objects stored within XIF files.
## the first IFD is special in that it does not contain image of mask information but general information about the file.
IFDs <- getIFD(fileName = test_data, offsets = sub_offs)

## 4) Extract Images and Masks

objectExtract(ifd = IFDs, info = metadata, mode = "gray",
              export = "file",
              write_to = "%d/%s/%s_%o_%c.tiff",
              overwrite = TRUE,
              selection = "all",
              force_range = TRUE)


ExtractMasks_toFile(raw, objects = sel, offsets = cif_offs, force_range = TRUE,
                    write_to = "%d/%s/%s_%o_%c.tiff",
                    base64_id = TRUE, add_noise = FALSE)

## Step 2) Create the multichannel images/mask and import them into OMERO using ezomero

### Connect to OMERO

In [ ]:
OMEROUSER = input(f"Enter username: \t")
OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")

OMEROHOST = "localhost"
OMEROPORT = "4064"

# Connection Check:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

## Information about the connection and its status
print(conn.isConnected())
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

### Create the multichannel images, import them into OMERO and associate masks

In [ ]:
files_path = input(f"Images file path: \t")
path = input(f"Mask file path: \t")
channels = input(f"Number of channels: \t")
metadata_path = input(f"Metadata path: \t")
files_output = input(f"Name of the folder where to export multichannel images: \t")


## Create the multichannel Images
multi_channel_conversion(files_path, metadata_path, files_output)
## Create a new dataset
dts = ezomero.post_dataset(conn, "Processed", project_id=1)
## push the image with REMBI annotation for analyzed data
ezomero.ezimport(conn, files_output, dataset= dts)
## Associate masks
associate_mask(conn, dts, path, channels)